In [9]:
import pandas as pd

In [10]:
common_characters_df = pd.read_csv('./data/common_chinese_characters.csv', index_col=0)
common_characters_df.head()

,汉字,常用
ID,,
1,一,1
2,乙,1
3,二,1
4,十,1
5,丁,1


In [11]:
common_characters_df = common_characters_df[common_characters_df['常用'] == 1]
common_characters = common_characters_df['汉字'].tolist()

# Generated Chinese Character Dataset